## **Librerías**

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_columns = False

In [3]:
from datetime import datetime

In [59]:
import matplotlib.pyplot as plt
import seaborn as sns

## **Datos**

In [4]:
df_retail = pd.read_csv('../Data/online_retail.csv')

In [6]:
df_retail.sample()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
29869,491956,22128,PARTY CONES CANDY ASSORTED,6,2009-12-14 16:00:00,1.25,16985.0,United Kingdom


In [9]:
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      525461 non-null  object 
 1   StockCode    525461 non-null  object 
 2   Description  522533 non-null  object 
 3   Quantity     525461 non-null  int64  
 4   InvoiceDate  525461 non-null  object 
 5   Price        525461 non-null  float64
 6   Customer ID  417534 non-null  float64
 7   Country      525461 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.1+ MB


## **Feature Engineering**

### **Limpieza de Columnas**

In [10]:
# Ajustamos el nombre de algunas columnas
df_retail.rename(columns={
    'Customer ID': 'CustomerID'
}, inplace=True)

In [13]:
# Ajustamos el formato de nuestro id
df_retail['CustomerID'].fillna(0, inplace=True)
df_retail['CustomerID'] = df_retail['CustomerID'].apply(lambda x: int(x))

In [14]:
# Ajustamos el formato de nuestra fecha de factura
df_retail['InvoiceDate'] = df_retail['InvoiceDate'].apply(lambda x: pd.to_datetime(x))

In [16]:
df_retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom


### **Creación de Features**

In [34]:
# Creamos una variable de mes de facturación
df_retail['InvoiceMonth'] = df_retail['InvoiceDate'].apply(lambda x: datetime(x.year, x.month, x.day))

In [35]:
# Agrupamos a nuestros usuarios por mes de facturación
grouping = df_retail.groupby('CustomerID')['InvoiceMonth']

In [36]:
# Nos quedamos con el primer registro defacturación disponible
df_retail['CohortMonth'] = grouping.transform('min')

In [37]:
df_retail.sample()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,InvoiceMonth,CohortMonth
23114,491185,21035,SET/2 RED SPOTTY TEA TOWELS,4,2009-12-10 12:21:00,2.95,17377,United Kingdom,2009-12-10,2009-12-02


In [71]:
# Calculamos los días entre cada compra y la primera
df_retail['DateDiff'] = df_retail['InvoiceMonth'] - df_retail['CohortMonth']
df_retail['DateDiff'] = pd.to_numeric(df_retail['DateDiff'].dt.days, downcast='integer')
df_retail['DateDiff'] = df_retail['DateDiff'].apply(lambda x: int(x))

df_retail['DateDiffMonths'] = df_retail['DateDiff'].apply(lambda x: int(round(x/30) + 1))

In [72]:
df_retail['Cohort'] = df_retail['CohortMonth'].apply(lambda x: str(x)[:7])

In [75]:
df_retail.sample()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,InvoiceMonth,CohortMonth,DateDiff,DateDiffMonths,Cohort
466727,533374,22747,POPPY'S PLAYHOUSE BATHROOM,6,2010-11-17 10:34:00,2.1,14049,United Kingdom,2010-11-17,2010-04-01,230,9,2010-04


In [84]:
pd.crosstab(
    index=df_retail['Cohort'], 
    columns=df_retail['DateDiffMonths']
)

DateDiffMonths,1,2,3,4,5,6,7,8,9,10,11,12,13
Cohort,,,,,,,,,,,,,
2009-12,40723,16200,20995,21225,20023,20993,21373,19290,18105,16768,25705,29645,38378
2010-01,9599,2418,2712,2854,3367,2218,2653,2202,2860,3792,3628,1006,0
2010-02,9815,1614,2518,2585,2470,1900,1956,2645,3086,3294,964,0,0
2010-03,11858,2021,3129,3089,2581,2361,3012,3516,4420,1044,0,0,0
2010-04,6774,935,1068,985,1086,1385,2021,1861,516,0,0,0,0
2010-05,6527,856,689,690,657,2102,1765,570,0,0,0,0,0
2010-06,7646,1005,1151,1055,2454,2129,944,0,0,0,0,0,0
2010-07,5192,642,1127,1744,1787,711,0,0,0,0,0,0,0
2010-08,4717,1050,1208,1506,478,0,0,0,0,0,0,0,0
